In [136]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt

In [99]:
data = pd.read_csv('../Data/accident_data.csv')

/Users/mattlampl/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (19,20,73,77,78,79,80,81,82,83,84,183,189) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [100]:
location_data = data.dropna(subset=['LONGITUDE', 'LATITUDE'])

In [126]:
geometry = geometry = [Point(xy) for xy in zip(location_data['DEC_LONG'], location_data['DEC_LAT'])]
location_gdf = gpd.GeoDataFrame(location_data, geometry=geometry, crs='EPSG:4326')

In [127]:
# Import the roads file
roads = gpd.read_file('alleghenycounty_streetcenterlines202303.zip', crs="EPSG:4326")

In [188]:
acc_road_join = gpd.sjoin_nearest(location_gdf.to_crs(roads.crs), roads, how='left')

In [189]:
acc_road_agg = acc_road_join.groupby('FULL_NAME')['CRASH_CRN'].count()\
    .reset_index()\
    .rename(columns={'FULL_NAME':'STREET', 'CRASH_CRN':'COUNT'})\
    .sort_values('COUNT', ascending=False)
acc_road_agg.head(20)

,STREET,COUNT
3163,I376 EB,6808
3198,I376 WB,5966
5977,SAW MILL RUN BLVD,3045
4907,OHIO RIVER BLVD,2684
5221,PERRY HWY,2583
73,5TH AVE,2579
6329,STATE ROUTE 28 NB,2259
6347,STATE ROUTE 28 SB,2248
5195,PENN AVE,2073
7226,WILLIAM PENN HWY,2057


In [193]:
road_acc_counts = roads.merge(acc_road_agg, how='inner', left_on='FULL_NAME', right_on='STREET')